In [24]:
rm(list = ls())

library(data.table)
library(tidyverse)
#library(rJava)
#library(RNetLogo)

library(ggplot2)
library(plotly)  # For beautiful plotting
library(randomForest)
#library(factoextra)
#library(e1071)
#library(TSrepr)  # for evaluating predictive power
#library(lhs)  # For maximin Latin hypercube sampling
#library(caret)
require(gridExtra)
library(GGally)
library(zoo)

options(warn = -1)

# data sets

In [25]:
# specify training and test data folder
folder.path = "C:/Users/paslanpatir/Desktop/TEZ_v2/"
data.path = paste0(folder.path,"data/")

In [26]:
## Fixed Pool
unlabeled.1= fread(paste0(data.path,"unlabeled_pool","_basic_100.csv")) 
unlabeled.2= fread(paste0(data.path,"unlabeled_pool","_dummy_100.csv")) 

## Test sets
test_set_all.1 = data.table()
test_set_all.2 = data.table()
for( i in c(100,200,300,400)){    
    test_set.1 <- fread(paste0(data.path,"test_set","_","basic","_",i,".csv"))     
    test_set.2 <- fread(paste0(data.path,"test_set","_","dummy","_",i,".csv"))  
    test_set_all.1 = rbind(test_set_all.1,data.table(test_set.1, "size" = i))
    test_set_all.2 = rbind(test_set_all.2,data.table(test_set.2, "size" = i))    
    rm(test_set.1,test_set.2)    
}

## Training Sets
# Large LHS
training_set_all.1 = data.table()
training_set_all.2 = data.table()
# small LHS
training_set_all_small.1 = data.table()
training_set_all_small.2 = data.table()

for( i in c(0,1,2,3,4,5,6,7,8,9,20)){
    training_set.1 <- fread(paste0(data.path,"training_set","_","basic","_100_seed",i,".csv")) 
    training_set_all.1 = rbind(training_set_all.1,data.table(training_set.1, "seed" = i))
    training_set.2 <- fread(paste0(data.path,"training_set","_","dummy","_100_seed",i,".csv")) 
    training_set_all.2 = rbind(training_set_all.2,data.table(training_set.2, "seed" = i)) 
    
    rm(training_set.1,training_set.2)   
    
    training_set_small.1 <- fread(paste0(data.path,"training_set","_","basic","_50_seed",i,".csv")) 
    training_set_all_small.1 = rbind(training_set_all_small.1,data.table(training_set_small.1, "seed" = i))
    training_set_small.2 <- fread(paste0(data.path,"training_set","_","dummy","_50_seed",i,".csv")) 
    training_set_all_small.2 = rbind(training_set_all_small.2,data.table(training_set_small.2, "seed" = i)) 
    
    rm(training_set_small.1,training_set_small.2)   
}

# test_set_all.1, training_set_all.2
# test_set_all_small.1, training_set_all_small.2

# outputs

In [27]:
model.type.1 = "basic"
model.type.2 = "dummy"

feature_names.1 = c("density", "%-similar-wanted")
feature_names.2 = c("density", "%-similar-wanted", "budget-multiplier-dummy", "density-multiplier-dummy", "noise-dummy", "tick-limit")

## main output folders
outputs.path.1.refr = paste0(folder.path,"outputs/outputs_50_100/outputs_SampleRef_Basic","/")
    # child paths
    path.1.refr.ps         = paste0(outputs.path.1.refr,"oneshot/")
    path.1.refr.rd         = paste0(outputs.path.1.refr,"Rd/")
    path.1.refr.ad_rng     = paste0(outputs.path.1.refr,"Ad_range/")
    path.1.refr.ad_sd      = paste0(outputs.path.1.refr,"Ad_sd/")
    path.1.refr.ad_cvar    = paste0(outputs.path.1.refr,"Ad_coefvar/")
    path.1.refr.ad_cvar_6  = paste0(outputs.path.1.refr,"Ad_coefvar_mtry6/")

outputs.path.2.refr = paste0(folder.path,"outputs/outputs_50_100/outputs_SampleRef_Dummy","/")
    # child paths
    path.2.refr.ps               = paste0(outputs.path.2.refr,"oneshot/")
    path.2.refr.ps_3             = paste0(outputs.path.2.refr,"oneshot_mtry3/")
    path.2.refr.ps_4             = paste0(outputs.path.2.refr,"oneshot_mtry4/")
    path.2.refr.ps_6             = paste0(outputs.path.2.refr,"oneshot_mtry6/")

    path.2.refr.rd               = paste0(outputs.path.2.refr,"Rd/")
    path.2.refr.ad_rng           = paste0(outputs.path.2.refr,"Ad_range/")
    path.2.refr.ad_rng_4         = paste0(outputs.path.2.refr,"Ad_range_mtry4/")
    path.2.refr.ad_sd            = paste0(outputs.path.2.refr,"Ad_sd/")
    path.2.refr.ad_sd_3          = paste0(outputs.path.2.refr,"Ad_sd_mtry3/")
    path.2.refr.ad_sd_4          = paste0(outputs.path.2.refr,"Ad_sd_mtry4/")
    path.2.refr.ad_sd_6          = paste0(outputs.path.2.refr,"Ad_sd_mtry6/")

    path.2.refr.ad_cvar          = paste0(outputs.path.2.refr,"Ad_coefvar/")
    path.2.refr.ad_cvar_6        = paste0(outputs.path.2.refr,"Ad_coefvar_mtry6/")

    path.2.refr.adfe_begin_cvar  = paste0(outputs.path.2.refr,"Ad_coefvar_FeatSel_Begin/")
    path.2.refr.adfe_end_cvar    = paste0(outputs.path.2.refr,"Ad_coefvar_FeatSel_End/")


outputs.path.1.wrep = paste0(folder.path,"outputs/outputs_50_100/outputs_WRep_Basic","/")
    # child paths
    path.1.wrep.ps       = paste0(outputs.path.1.wrep,"oneshot/")
    path.1.wrep.rd       = paste0(outputs.path.1.wrep,"Rd/")
    path.1.wrep.ad_rng   = paste0(outputs.path.1.wrep,"Ad_range/")
    path.1.wrep.ad_sd    = paste0(outputs.path.1.wrep,"Ad_sd/")
    path.1.wrep.ad_cvar  = paste0(outputs.path.1.wrep,"Ad_coefvar/")

outputs.path.2.wrep = paste0(folder.path,"outputs/outputs_50_100/outputs_WRep_Dummy","/")
    # child paths
    path.2.wrep.ps       = paste0(outputs.path.2.wrep,"oneshot/")
    path.2.wrep.rd       = paste0(outputs.path.2.wrep,"Rd/")
    path.2.wrep.ad_rng   = paste0(outputs.path.2.wrep,"Ad_range/")
    path.2.wrep.ad_sd    = paste0(outputs.path.2.wrep,"Ad_sd/")
    path.2.wrep.ad_cvar  = paste0(outputs.path.2.wrep,"Ad_coefvar/")


# test performance data

In [9]:
# wrep
perf.wrep.1.ps        = fread(paste0(path.1.wrep.ps,model.type.1,"_performance_table_oneshot",".csv"))
perf.wrep.1.ps        = perf.wrep.1.ps[size == 100]
perf.wrep.1.ps[,size := NULL]

perf.wrep.1.rd        = fread(paste0(path.1.wrep.rd,model.type.1,"_performance_table_Rd",".csv"))
perf.wrep.1.ad_rng    = fread(paste0(path.1.wrep.ad_rng,model.type.1,"_performance_table_Ad",".csv"))
perf.wrep.1.ad_sd     = fread(paste0(path.1.wrep.ad_sd,model.type.1,"_performance_table_Ad",".csv"))
perf.wrep.1.ad_cvar   = fread(paste0(path.1.wrep.ad_cvar,model.type.1,"_performance_table_Ad",".csv"))

perf.wrep.2.ps        = fread(paste0(path.2.wrep.ps,model.type.2,"_performance_table_oneshot",".csv"))
perf.wrep.2.ps        = perf.wrep.2.ps[size == 100]
perf.wrep.2.ps[,size := NULL]
perf.wrep.2.rd        = fread(paste0(path.2.wrep.rd,model.type.2,"_performance_table_Rd",".csv"))
perf.wrep.2.ad_rng    = fread(paste0(path.2.wrep.ad_rng,model.type.2,"_performance_table_Ad",".csv"))
perf.wrep.2.ad_sd     = fread(paste0(path.2.wrep.ad_sd,model.type.2,"_performance_table_Ad",".csv"))
perf.wrep.2.ad_cvar   = fread(paste0(path.2.wrep.ad_cvar,model.type.2,"_performance_table_Ad",".csv"))

perf.wrep = rbind(data.table(perf.wrep.1.ps     ,"model" = "basic",pool = "wrep", "sampling" = "ps",     "testsize" = 100, "mtry" = 2),
                  data.table(perf.wrep.1.rd     ,"model" = "basic",pool = "wrep", "sampling" = "rd",     "testsize" = 100, "mtry" = 2),
                  data.table(perf.wrep.1.ad_rng ,"model" = "basic",pool = "wrep", "sampling" = "ad_rng", "testsize" = 100, "mtry" = 2),
                  data.table(perf.wrep.1.ad_sd  ,"model" = "basic",pool = "wrep", "sampling" = "ad_sd",  "testsize" = 100, "mtry" = 2),
                  data.table(perf.wrep.1.ad_cvar,"model" = "basic",pool = "wrep", "sampling" = "ad_cvar","testsize" = 100, "mtry" = 2),
                  data.table(perf.wrep.2.ps     ,"model" = "dummy",pool = "wrep", "sampling" = "ps"    , "testsize" = 100, "mtry" = 2),
                  data.table(perf.wrep.2.rd     ,"model" = "dummy",pool = "wrep", "sampling" = "rd"    , "testsize" = 100, "mtry" = 2),
                  data.table(perf.wrep.2.ad_rng ,"model" = "dummy",pool = "wrep", "sampling" = "ad_rng", "testsize" = 100, "mtry" = 2),
                  data.table(perf.wrep.2.ad_sd  ,"model" = "dummy",pool = "wrep", "sampling" = "ad_sd" , "testsize" = 100, "mtry" = 2),
                  data.table(perf.wrep.2.ad_cvar,"model" = "dummy",pool = "wrep", "sampling" = "ad_cvar","testsize" = 100, "mtry" = 2)                 
)

rm(perf.wrep.1.ps      
  ,perf.wrep.1.rd      
  ,perf.wrep.1.ad_rng  
  ,perf.wrep.1.ad_sd   
  ,perf.wrep.1.ad_cvar 
  ,perf.wrep.2.ps      
  ,perf.wrep.2.rd      
  ,perf.wrep.2.ad_rng  
  ,perf.wrep.2.ad_sd   
  ,perf.wrep.2.ad_cvar )

In [10]:
# refr
perf.refr.1.ps               = fread(paste0(path.1.refr.ps         ,model.type.1,"_performance_table_oneshot",".csv"))
perf.refr.1.rd               = fread(paste0(path.1.refr.rd         ,model.type.1,"_performance_table_Rd",".csv"))
perf.refr.1.ad_rng           = fread(paste0(path.1.refr.ad_rng     ,model.type.1,"_performance_table_Ad_range",".csv"))
perf.refr.1.ad_sd            = fread(paste0(path.1.refr.ad_sd      ,model.type.1,"_performance_table_Ad",".csv"))
perf.refr.1.ad_cvar          = fread(paste0(path.1.refr.ad_cvar    ,model.type.1,"_performance_table_Ad_coefvar",".csv"))

perf.refr.2.ps               = fread(paste0(path.2.refr.ps         ,model.type.2,"_performance_table_oneshot",".csv"))
perf.refr.2.ps_3             = fread(paste0(path.2.refr.ps_3       ,model.type.2,"_performance_table_oneshot",".csv"))
perf.refr.2.ps_4             = fread(paste0(path.2.refr.ps_4       ,model.type.2,"_performance_table_oneshot",".csv"))
perf.refr.2.ps_6             = fread(paste0(path.2.refr.ps_6       ,model.type.2,"_performance_table_oneshot",".csv"))
perf.refr.2.rd               = fread(paste0(path.2.refr.rd         ,model.type.2,"_performance_table_Rd",".csv"))
perf.refr.2.ad_rng           = fread(paste0(path.2.refr.ad_rng     ,model.type.2,"_performance_table_Ad_range",".csv"))
perf.refr.2.ad_rng_4         = fread(paste0(path.2.refr.ad_rng_4   ,model.type.2,"_performance_table_Ad_range",".csv"))
perf.refr.2.ad_sd            = fread(paste0(path.2.refr.ad_sd      ,model.type.2,"_performance_table_Ad",".csv"))
perf.refr.2.ad_sd_3          = fread(paste0(path.2.refr.ad_sd_3    ,model.type.2,"_performance_table_Ad_sd",".csv"))
perf.refr.2.ad_sd_4          = fread(paste0(path.2.refr.ad_sd_4    ,model.type.2,"_performance_table_Ad_sd",".csv"))
perf.refr.2.ad_sd_6          = fread(paste0(path.2.refr.ad_sd_6    ,model.type.2,"_performance_table_Ad_sd",".csv"))
perf.refr.2.ad_cvar          = fread(paste0(path.2.refr.ad_cvar    ,model.type.2,"_performance_table_Ad_coefvar",".csv"))
perf.refr.2.ad_cvar_6        = fread(paste0(path.2.refr.ad_cvar_6  ,model.type.2,"_performance_table_Ad_coefvar",".csv"))
perf.refr.2.adfe_begin_cvar  = fread(paste0(path.2.refr.adfe_begin_cvar  ,model.type.2,"_performance_table_AdFe_coefvar",".csv"))
perf.refr.2.adfe_end_cvar    = fread(paste0(path.2.refr.adfe_end_cvar    ,model.type.2,"_performance_table_AdFe_coefvar",".csv"))

perf.refr = rbind(data.table(perf.refr.1.ps              ,"model" = "basic", "pool" = "refr", "sampling" = "ps"       ,"mtry" = 2),
                  data.table(perf.refr.1.rd              ,"model" = "basic", "pool" = "refr", "sampling" = "rd"       ,"mtry" = 2),
                  data.table(perf.refr.1.ad_rng          ,"model" = "basic", "pool" = "refr", "sampling" = "ad_rng"   ,"mtry" = 2),
                  data.table(perf.refr.1.ad_sd           ,"model" = "basic", "pool" = "refr", "sampling" = "ad_sd"    ,"mtry" = 2),
                  data.table(perf.refr.1.ad_cvar         ,"model" = "basic", "pool" = "refr", "sampling" = "ad_cvar"  ,"mtry" = 2),
                  data.table(perf.refr.2.ps              ,"model" = "dummy", "pool" = "refr", "sampling" = "ps"       ,"mtry" = 2),
                  data.table(perf.refr.2.ps_3            ,"model" = "dummy", "pool" = "refr", "sampling" = "ps"       ,"mtry" = 3),
                  data.table(perf.refr.2.ps_4            ,"model" = "dummy", "pool" = "refr", "sampling" = "ps"       ,"mtry" = 4),
                  data.table(perf.refr.2.ps_6            ,"model" = "dummy", "pool" = "refr", "sampling" = "ps"       ,"mtry" = 6),     
                  data.table(perf.refr.2.rd              ,"model" = "dummy", "pool" = "refr", "sampling" = "rd"       ,"mtry" = 2),
                  data.table(perf.refr.2.ad_rng          ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_rng"   ,"mtry" = 2),
                  data.table(perf.refr.2.ad_rng_4        ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_rng"   ,"mtry" = 4),
                  data.table(perf.refr.2.ad_sd           ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_sd"    ,"mtry" = 2),
                  data.table(perf.refr.2.ad_sd_3         ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_sd"    ,"mtry" = 3),
                  data.table(perf.refr.2.ad_sd_4         ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_sd"    ,"mtry" = 4),
                  data.table(perf.refr.2.ad_sd_6         ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_sd"    ,"mtry" = 6),
                  data.table(perf.refr.2.ad_cvar         ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_cvar"  ,"mtry" = 2),
                  data.table(perf.refr.2.ad_cvar_6       ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_cvar"  ,"mtry" = 6),
                  data.table(perf.refr.2.adfe_begin_cvar ,"model" = "dummy", "pool" = "refr", "sampling" = "adfe_begin_cvar","mtry" = 2),
                  data.table(perf.refr.2.adfe_end_cvar   ,"model" = "dummy", "pool" = "refr", "sampling" = "adfe_end_cvar"  ,"mtry" = 2)                             
)
setnames(perf.refr,"size","testsize")

rm(perf.refr.1.ps             
,perf.refr.1.rd             
,perf.refr.1.ad_rng         
,perf.refr.1.ad_sd          
,perf.refr.1.ad_cvar        
,perf.refr.1.ad_cvar_6      
,perf.refr.2.ps             
,perf.refr.2.ps_3           
,perf.refr.2.ps_4           
,perf.refr.2.ps_6           
,perf.refr.2.rd             
,perf.refr.2.ad_rng         
,perf.refr.2.ad_rng_4       
,perf.refr.2.ad_sd          
,perf.refr.2.ad_sd_3        
,perf.refr.2.ad_sd_4        
,perf.refr.2.ad_sd_6        
,perf.refr.2.ad_cvar        
,perf.refr.2.ad_cvar_6      
,perf.refr.2.adfe_begin_cvar
,perf.refr.2.adfe_end_cvar)  

In [11]:
perf_all = rbind(perf.wrep,perf.refr, use.names = TRUE)
rm(perf.wrep,perf.refr)

# oob error data

In [12]:
# wrep
oob.wrep.1.ps        = fread(paste0(path.1.wrep.ps,model.type.1,"_obb_error_oneshot_all",".csv"))
oob.wrep.1.rd        = fread(paste0(path.1.wrep.rd,model.type.1,"_obb_error_Rd",".csv"))
oob.wrep.1.ad_rng    = fread(paste0(path.1.wrep.ad_rng,model.type.1,"_obb_error_Ad",".csv"))
oob.wrep.1.ad_sd     = fread(paste0(path.1.wrep.ad_sd,model.type.1,"_obb_error_Ad",".csv"))
oob.wrep.1.ad_cvar   = fread(paste0(path.1.wrep.ad_cvar,model.type.1,"_obb_error_Ad",".csv"))

oob.wrep.2.ps        = fread(paste0(path.2.wrep.ps,model.type.2,"_obb_error_oneshot_all",".csv"))
oob.wrep.2.rd        = fread(paste0(path.2.wrep.rd,model.type.2,"_obb_error_Rd",".csv"))
oob.wrep.2.ad_rng    = fread(paste0(path.2.wrep.ad_rng,model.type.2,"_obb_error_Ad",".csv"))
oob.wrep.2.ad_sd     = fread(paste0(path.2.wrep.ad_sd,model.type.2,"_obb_error_Ad",".csv"))
oob.wrep.2.ad_cvar   = fread(paste0(path.2.wrep.ad_cvar,model.type.2,"_obb_error_Ad",".csv"))

oob.wrep = rbind( data.table("iter" = 1 ,oob.wrep.1.ps     ,"model" = "basic",pool = "wrep", "sampling" = "ps",      "mtry" = 2),
                  data.table(oob.wrep.1.rd     ,"model" = "basic",pool = "wrep", "sampling" = "rd",      "mtry" = 2),
                  data.table(oob.wrep.1.ad_rng ,"model" = "basic",pool = "wrep", "sampling" = "ad_rng",  "mtry" = 2),
                  data.table(oob.wrep.1.ad_sd  ,"model" = "basic",pool = "wrep", "sampling" = "ad_sd",   "mtry" = 2),
                  data.table(oob.wrep.1.ad_cvar,"model" = "basic",pool = "wrep", "sampling" = "ad_cvar", "mtry" = 2),
                  data.table("iter" = 1 ,oob.wrep.2.ps     ,"model" = "dummy",pool = "wrep", "sampling" = "ps"    ,  "mtry" = 2),
                  data.table(oob.wrep.2.rd     ,"model" = "dummy",pool = "wrep", "sampling" = "rd"    ,  "mtry" = 2),
                  data.table(oob.wrep.2.ad_rng ,"model" = "dummy",pool = "wrep", "sampling" = "ad_rng",  "mtry" = 2),
                  data.table(oob.wrep.2.ad_sd  ,"model" = "dummy",pool = "wrep", "sampling" = "ad_sd" ,  "mtry" = 2),
                  data.table(oob.wrep.2.ad_cvar,"model" = "dummy",pool = "wrep", "sampling" = "ad_cvar", "mtry" = 2)                 
)

rm(oob.wrep.1.ps      
  ,oob.wrep.1.rd      
  ,oob.wrep.1.ad_rng  
  ,oob.wrep.1.ad_sd   
  ,oob.wrep.1.ad_cvar 
  ,oob.wrep.2.ps      
  ,oob.wrep.2.rd      
  ,oob.wrep.2.ad_rng  
  ,oob.wrep.2.ad_sd   
  ,oob.wrep.2.ad_cvar )

In [13]:
# refr
oob.refr.1.ps               = fread(paste0(path.1.refr.ps         ,model.type.1,"_obb_error_oneshot_all",".csv"))
oob.refr.1.rd               = fread(paste0(path.1.refr.rd         ,model.type.1,"_obb_error_Rd",".csv"))
oob.refr.1.ad_rng           = fread(paste0(path.1.refr.ad_rng     ,model.type.1,"_obb_error_Ad_range",".csv"))
oob.refr.1.ad_sd            = fread(paste0(path.1.refr.ad_sd      ,model.type.1,"_obb_error_Ad",".csv"))
oob.refr.1.ad_cvar          = fread(paste0(path.1.refr.ad_cvar    ,model.type.1,"_obb_error_Ad_coefvar",".csv"))

oob.refr.2.ps               = fread(paste0(path.2.refr.ps         ,model.type.2,"_obb_error_oneshot_all",".csv"))
oob.refr.2.ps_3             = fread(paste0(path.2.refr.ps_3       ,model.type.2,"_obb_error_oneshot_all",".csv"))
oob.refr.2.ps_4             = fread(paste0(path.2.refr.ps_4       ,model.type.2,"_obb_error_oneshot_all",".csv"))
oob.refr.2.ps_6             = fread(paste0(path.2.refr.ps_6       ,model.type.2,"_obb_error_oneshot_all",".csv"))
oob.refr.2.rd               = fread(paste0(path.2.refr.rd         ,model.type.2,"_obb_error_Rd",".csv"))
oob.refr.2.ad_rng           = fread(paste0(path.2.refr.ad_rng     ,model.type.2,"_obb_error_Ad_range",".csv"))
oob.refr.2.ad_rng_4         = fread(paste0(path.2.refr.ad_rng_4   ,model.type.2,"_obb_error_Ad_range",".csv"))
oob.refr.2.ad_sd            = fread(paste0(path.2.refr.ad_sd      ,model.type.2,"_obb_error_Ad",".csv"))
oob.refr.2.ad_sd_3          = fread(paste0(path.2.refr.ad_sd_3    ,model.type.2,"_obb_error_Ad_sd",".csv"))
oob.refr.2.ad_sd_4          = fread(paste0(path.2.refr.ad_sd_4    ,model.type.2,"_obb_error_Ad_sd",".csv"))
oob.refr.2.ad_sd_6          = fread(paste0(path.2.refr.ad_sd_6    ,model.type.2,"_obb_error_Ad_sd",".csv"))
oob.refr.2.ad_cvar          = fread(paste0(path.2.refr.ad_cvar    ,model.type.2,"_obb_error_Ad_coefvar",".csv"))
oob.refr.2.ad_cvar_6        = fread(paste0(path.2.refr.ad_cvar_6  ,model.type.2,"_obb_error_Ad_coefvar",".csv"))
oob.refr.2.adfe_begin_cvar  = fread(paste0(path.2.refr.adfe_begin_cvar  ,model.type.2,"_obb_error_AdFe_coefvar",".csv"))
oob.refr.2.adfe_end_cvar    = fread(paste0(path.2.refr.adfe_end_cvar    ,model.type.2,"_obb_error_AdFe_coefvar",".csv"))

oob.refr = rbind( data.table("iter" = 1 ,oob.refr.1.ps              ,"model" = "basic", "pool" = "refr", "sampling" = "ps"       ,"mtry" = 2),
                  data.table(oob.refr.1.rd              ,"model" = "basic", "pool" = "refr", "sampling" = "rd"       ,"mtry" = 2),
                  data.table(oob.refr.1.ad_rng          ,"model" = "basic", "pool" = "refr", "sampling" = "ad_rng"   ,"mtry" = 2),
                  data.table(oob.refr.1.ad_sd           ,"model" = "basic", "pool" = "refr", "sampling" = "ad_sd"    ,"mtry" = 2),
                  data.table(oob.refr.1.ad_cvar         ,"model" = "basic", "pool" = "refr", "sampling" = "ad_cvar"  ,"mtry" = 2),
                  data.table("iter" = 1 ,oob.refr.2.ps              ,"model" = "dummy", "pool" = "refr", "sampling" = "ps"       ,"mtry" = 2),
                  data.table("iter" = 1 ,oob.refr.2.ps_3            ,"model" = "dummy", "pool" = "refr", "sampling" = "ps"       ,"mtry" = 3),
                  data.table("iter" = 1 ,oob.refr.2.ps_4            ,"model" = "dummy", "pool" = "refr", "sampling" = "ps"       ,"mtry" = 4),
                  data.table("iter" = 1 ,oob.refr.2.ps_6            ,"model" = "dummy", "pool" = "refr", "sampling" = "ps"       ,"mtry" = 6),     
                  data.table(oob.refr.2.rd              ,"model" = "dummy", "pool" = "refr", "sampling" = "rd"       ,"mtry" = 2),
                  data.table(oob.refr.2.ad_rng          ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_rng"   ,"mtry" = 2),
                  data.table(oob.refr.2.ad_rng_4        ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_rng"   ,"mtry" = 4),
                  data.table(oob.refr.2.ad_sd           ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_sd"    ,"mtry" = 2),
                  data.table(oob.refr.2.ad_sd_3         ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_sd"    ,"mtry" = 3),
                  data.table(oob.refr.2.ad_sd_4         ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_sd"    ,"mtry" = 4),
                  data.table(oob.refr.2.ad_sd_6         ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_sd"    ,"mtry" = 6),
                  data.table(oob.refr.2.ad_cvar         ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_cvar"  ,"mtry" = 2),
                  data.table(oob.refr.2.ad_cvar_6       ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_cvar"  ,"mtry" = 6),
                  data.table(oob.refr.2.adfe_begin_cvar ,"model" = "dummy", "pool" = "refr", "sampling" = "adfe_begin_cvar","mtry" = 2),
                  data.table(oob.refr.2.adfe_end_cvar   ,"model" = "dummy", "pool" = "refr", "sampling" = "adfe_end_cvar"  ,"mtry" = 2)                             
)

rm(
 oob.refr.1.ps             
,oob.refr.1.rd             
,oob.refr.1.ad_rng         
,oob.refr.1.ad_sd          
,oob.refr.1.ad_cvar        
,oob.refr.1.ad_cvar_6      
,oob.refr.2.ps             
,oob.refr.2.ps_3           
,oob.refr.2.ps_4           
,oob.refr.2.ps_6           
,oob.refr.2.rd             
,oob.refr.2.ad_rng         
,oob.refr.2.ad_rng_4       
,oob.refr.2.ad_sd          
,oob.refr.2.ad_sd_3        
,oob.refr.2.ad_sd_4        
,oob.refr.2.ad_sd_6        
,oob.refr.2.ad_cvar        
,oob.refr.2.ad_cvar_6      
,oob.refr.2.adfe_begin_cvar
,oob.refr.2.adfe_end_cvar)  

In [14]:
oob_all = rbind(oob.wrep,oob.refr, use.names = TRUE)
rm(oob.wrep,oob.refr)
oob_all[,iter := iter - 1]

# train candidate data

In [15]:
# wrep
candidate.wrep.1.rd        = fread(paste0(path.1.wrep.rd,model.type.1,"_train_candidates_table_Rd",".csv"))
candidate.wrep.1.ad_rng    = fread(paste0(path.1.wrep.ad_rng,model.type.1,"_train_candidates_table_Ad",".csv"))
candidate.wrep.1.ad_sd     = fread(paste0(path.1.wrep.ad_sd,model.type.1,"_train_candidates_table_Ad",".csv"))
candidate.wrep.1.ad_cvar   = fread(paste0(path.1.wrep.ad_cvar,model.type.1,"_train_candidates_table_Ad",".csv"))

candidate.wrep.2.rd        = fread(paste0(path.2.wrep.rd,model.type.2,"_train_candidates_table_Rd",".csv"))
candidate.wrep.2.ad_rng    = fread(paste0(path.2.wrep.ad_rng,model.type.2,"_train_candidates_table_Ad",".csv"))
candidate.wrep.2.ad_sd     = fread(paste0(path.2.wrep.ad_sd,model.type.2,"_train_candidates_table_Ad",".csv"))
candidate.wrep.2.ad_cvar   = fread(paste0(path.2.wrep.ad_cvar,model.type.2,"_train_candidates_table_Ad",".csv"))

candidate.wrep.1 = rbind( data.table(candidate.wrep.1.rd     ,"model" = "basic",pool = "wrep", "sampling" = "rd",      "mtry" = 2),
                          data.table(candidate.wrep.1.ad_rng ,"model" = "basic",pool = "wrep", "sampling" = "ad_rng",  "mtry" = 2),
                          data.table(candidate.wrep.1.ad_sd  ,"model" = "basic",pool = "wrep", "sampling" = "ad_sd",   "mtry" = 2),
                          data.table(candidate.wrep.1.ad_cvar,"model" = "basic",pool = "wrep", "sampling" = "ad_cvar", "mtry" = 2)
 )

candidate.wrep.2 = rbind( data.table(candidate.wrep.2.rd     ,"model" = "dummy",pool = "wrep", "sampling" = "rd"    ,  "mtry" = 2),
                          data.table(candidate.wrep.2.ad_rng ,"model" = "dummy",pool = "wrep", "sampling" = "ad_rng",  "mtry" = 2),
                          data.table(candidate.wrep.2.ad_sd  ,"model" = "dummy",pool = "wrep", "sampling" = "ad_sd" ,  "mtry" = 2),
                          data.table(candidate.wrep.2.ad_cvar,"model" = "dummy",pool = "wrep", "sampling" = "ad_cvar", "mtry" = 2)
)                         

rm( 
candidate.wrep.1.rd     
,candidate.wrep.1.ad_rng 
,candidate.wrep.1.ad_sd  
,candidate.wrep.1.ad_cvar
,candidate.wrep.2.rd     
,candidate.wrep.2.ad_rng 
,candidate.wrep.2.ad_sd  
,candidate.wrep.2.ad_cvar
)
   

In [16]:
# refr
candidate.refr.1.rd               = fread(paste0(path.1.refr.rd         ,model.type.1,"_train_candidates_table_Rd",".csv"))
candidate.refr.1.ad_rng           = fread(paste0(path.1.refr.ad_rng     ,model.type.1,"_train_candidates_table_Ad_range",".csv"))
candidate.refr.1.ad_sd            = fread(paste0(path.1.refr.ad_sd      ,model.type.1,"_train_candidates_table_Ad",".csv"))
candidate.refr.1.ad_cvar          = fread(paste0(path.1.refr.ad_cvar    ,model.type.1,"_train_candidates_table_Ad_coefvar",".csv"))

candidate.refr.2.rd               = fread(paste0(path.2.refr.rd         ,model.type.2,"_train_candidates_table_Rd",".csv"))
candidate.refr.2.ad_rng           = fread(paste0(path.2.refr.ad_rng     ,model.type.2,"_train_candidates_table_Ad_range",".csv"))
candidate.refr.2.ad_rng_4         = fread(paste0(path.2.refr.ad_rng_4   ,model.type.2,"_train_candidates_table_Ad_range",".csv"))
candidate.refr.2.ad_sd            = fread(paste0(path.2.refr.ad_sd      ,model.type.2,"_train_candidates_table_Ad",".csv"))
candidate.refr.2.ad_sd_3          = fread(paste0(path.2.refr.ad_sd_3    ,model.type.2,"_train_candidates_table_Ad_sd",".csv"))
candidate.refr.2.ad_sd_4          = fread(paste0(path.2.refr.ad_sd_4    ,model.type.2,"_train_candidates_table_Ad_sd",".csv"))
candidate.refr.2.ad_sd_6          = fread(paste0(path.2.refr.ad_sd_6    ,model.type.2,"_train_candidates_table_Ad_sd",".csv"))
candidate.refr.2.ad_cvar          = fread(paste0(path.2.refr.ad_cvar    ,model.type.2,"_train_candidates_table_Ad_coefvar",".csv"))
candidate.refr.2.ad_cvar_6        = fread(paste0(path.2.refr.ad_cvar_6  ,model.type.2,"_train_candidates_table_Ad_coefvar",".csv"))
candidate.refr.2.adfe_begin_cvar  = fread(paste0(path.2.refr.adfe_begin_cvar  ,model.type.2,"_train_candidates_table_AdFe_coefvar",".csv"))
candidate.refr.2.adfe_end_cvar    = fread(paste0(path.2.refr.adfe_end_cvar    ,model.type.2,"_train_candidates_table_AdFe_coefvar",".csv"))

candidate.refr.1 = rbind( data.table(candidate.refr.1.rd              ,"model" = "basic", "pool" = "refr", "sampling" = "rd"       ,"mtry" = 2),
                          data.table(candidate.refr.1.ad_rng          ,"model" = "basic", "pool" = "refr", "sampling" = "ad_rng"   ,"mtry" = 2),
                          data.table(candidate.refr.1.ad_sd           ,"model" = "basic", "pool" = "refr", "sampling" = "ad_sd"    ,"mtry" = 2),
                          data.table(candidate.refr.1.ad_cvar         ,"model" = "basic", "pool" = "refr", "sampling" = "ad_cvar"  ,"mtry" = 2)
)                        
candidate.refr.2 = rbind( data.table(candidate.refr.2.rd              ,"model" = "dummy", "pool" = "refr", "sampling" = "rd"       ,"mtry" = 2),
                          data.table(candidate.refr.2.ad_rng          ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_rng"   ,"mtry" = 2),
                          data.table(candidate.refr.2.ad_rng_4        ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_rng"   ,"mtry" = 4),
                          data.table(candidate.refr.2.ad_sd           ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_sd"    ,"mtry" = 2),
                          data.table(candidate.refr.2.ad_sd_3         ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_sd"    ,"mtry" = 3),
                          data.table(candidate.refr.2.ad_sd_4         ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_sd"    ,"mtry" = 4),
                          data.table(candidate.refr.2.ad_sd_6         ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_sd"    ,"mtry" = 6),
                          data.table(candidate.refr.2.ad_cvar         ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_cvar"  ,"mtry" = 2),
                          data.table(candidate.refr.2.ad_cvar_6       ,"model" = "dummy", "pool" = "refr", "sampling" = "ad_cvar"  ,"mtry" = 6),
                          data.table(candidate.refr.2.adfe_begin_cvar ,"model" = "dummy", "pool" = "refr", "sampling" = "adfe_begin_cvar","mtry" = 2),
                          data.table(candidate.refr.2.adfe_end_cvar   ,"model" = "dummy", "pool" = "refr", "sampling" = "adfe_end_cvar"  ,"mtry" = 2)                             
)

rm(
candidate.refr.1.rd             
,candidate.refr.1.ad_rng         
,candidate.refr.1.ad_sd          
,candidate.refr.1.ad_cvar        
,candidate.refr.2.rd             
,candidate.refr.2.ad_rng         
,candidate.refr.2.ad_rng_4       
,candidate.refr.2.ad_sd          
,candidate.refr.2.ad_sd_3        
,candidate.refr.2.ad_sd_4        
,candidate.refr.2.ad_sd_6        
,candidate.refr.2.ad_cvar        
,candidate.refr.2.ad_cvar_6      
,candidate.refr.2.adfe_begin_cvar
,candidate.refr.2.adfe_end_cvar             
)  

In [17]:
candidate.all.1 = rbind(candidate.refr.1,candidate.wrep.1)
candidate.all.2 = rbind(candidate.refr.2,candidate.wrep.2)
rm(candidate.refr.1,candidate.refr.2,candidate.wrep.1,candidate.wrep.2)

# importance data

In [18]:
# wrep
importance.wrep.1.ad_rng    = fread(paste0(path.1.wrep.ad_rng,model.type.1,"_importance_table_Ad",".csv"))
importance.wrep.1.ad_sd     = fread(paste0(path.1.wrep.ad_sd,model.type.1,"_importance_table_Ad",".csv"))
importance.wrep.1.ad_cvar   = fread(paste0(path.1.wrep.ad_cvar,model.type.1,"_importance_table_Ad",".csv"))

importance.wrep.2.ad_rng    = fread(paste0(path.2.wrep.ad_rng,model.type.2,"_importance_table_Ad",".csv"))
importance.wrep.2.ad_sd     = fread(paste0(path.2.wrep.ad_sd,model.type.2,"_importance_table_Ad",".csv"))
importance.wrep.2.ad_cvar   = fread(paste0(path.2.wrep.ad_cvar,model.type.2,"_importance_table_Ad",".csv"))


importance.wrep.1 = rbind(data.table(importance.wrep.1.ad_rng ,"model" = "basic",pool = "wrep", "sampling" = "ad_rng",  "mtry" = 2),
                          data.table(importance.wrep.1.ad_sd  ,"model" = "basic",pool = "wrep", "sampling" = "ad_sd",   "mtry" = 2),
                          data.table(importance.wrep.1.ad_cvar,"model" = "basic",pool = "wrep", "sampling" = "ad_cvar", "mtry" = 2)
 )

importance.wrep.2 = rbind(data.table(importance.wrep.2.ad_rng ,"model" = "dummy",pool = "wrep", "sampling" = "ad_rng",  "mtry" = 2),
                          data.table(importance.wrep.2.ad_sd  ,"model" = "dummy",pool = "wrep", "sampling" = "ad_sd" ,  "mtry" = 2),
                          data.table(importance.wrep.2.ad_cvar,"model" = "dummy",pool = "wrep", "sampling" = "ad_cvar", "mtry" = 2)
)                         

rm( 
importance.wrep.1.ad_rng 
,importance.wrep.1.ad_sd  
,importance.wrep.1.ad_cvar
,importance.wrep.2.ad_rng 
,importance.wrep.2.ad_sd  
,importance.wrep.2.ad_cvar
)

importance.wrep.1 = melt(importance.wrep.1, id.vars = c("seed","rep","iter_no","model","pool","sampling","mtry"), measure.vars = c("density","%-similar-wanted"))
setnames(importance.wrep.1, "variable","feature")

importance.wrep.2 = melt(importance.wrep.2, id.vars = c("seed","rep","iter_no","model","pool","sampling","mtry"), measure.vars = c("density","%-similar-wanted","budget-multiplier-dummy","density-multiplier-dummy","noise-dummy","tick-limit"))
setnames(importance.wrep.2, "variable","feature")

In [19]:
importance.wrep.all = rbind(importance.wrep.1,importance.wrep.2)
rm(importance.wrep.1,importance.wrep.2)

In [20]:
#refr
importance.refr.1.ps                 = fread(paste0(path.1.refr.ps   ,model.type.1,"_importance_table_oneshot",".csv"))
importance.refr.1.ad_cvar            = fread(paste0(path.1.refr.ad_cvar          ,model.type.1,"_recovered_importance_table_Ad_coefvar",".csv"))

importance.refr.2.ad_cvar            = fread(paste0(path.2.refr.ad_cvar          ,model.type.2,"_recovered_importance_table_Ad_coefvar",".csv"))
importance.refr.2.ad_cvar_6          = fread(paste0(path.2.refr.ad_cvar_6        ,model.type.2,"_recovered_importance_table_Ad_coefvar",".csv"))
importance.refr.2.adfe_begin_cvar    = fread(paste0(path.2.refr.adfe_begin_cvar  ,model.type.2,"_recovered_importance_table_AdFe_coefvar",".csv"))
importance.refr.2.adfe_end_cvar      = fread(paste0(path.2.refr.adfe_end_cvar    ,model.type.2,"_recovered_importance_table_AdFe_coefvar",".csv"))

importance.refr.2.ps     = fread(paste0(path.2.refr.ps   ,model.type.2,"_importance_table_oneshot",".csv"))
importance.refr.2.ps_3   = fread(paste0(path.2.refr.ps_3 ,model.type.2,"_importance_table_oneshot",".csv"))
importance.refr.2.ps_4   = fread(paste0(path.2.refr.ps_4 ,model.type.2,"_importance_table_oneshot",".csv"))
importance.refr.2.ps_6   = fread(paste0(path.2.refr.ps_6 ,model.type.2,"_importance_table_oneshot",".csv"))

importance.refr.2.ps = dcast(importance.refr.2.ps, seed+rep+iter_no+scaled+type~feature, value.var = "value") 

importance.refr.2.ps = rbind(data.table(importance.refr.2.ps,"mtry" = 2),
                             data.table(importance.refr.2.ps_3,"mtry" = 3),
                             data.table(importance.refr.2.ps_4,"mtry" = 4),
                             data.table(importance.refr.2.ps_6,"mtry" = 6), use.names = TRUE
)
rm(importance.refr.2.ps_3,importance.refr.2.ps_4,importance.refr.2.ps_6)

In [21]:
importance.refr.1.ps[,mtry := 2]

importance.refr.2.ps = melt(importance.refr.2.ps, id.vars = c("seed","rep","iter_no","scaled","type","mtry"), measure.vars = c("density","%-similar-wanted","budget-multiplier-dummy","density-multiplier-dummy","noise-dummy","tick-limit"))
setnames(importance.refr.2.ps, "variable","feature")

In [22]:
importance.refr.all = rbind(data.table(importance.refr.1.ps                ,"model" = "basic",pool = "refr", "sampling" = "ps"),
                            data.table(importance.refr.1.ad_cvar           ,"model" = "basic",pool = "refr", "sampling" = "ad_cvar", "mtry" = 2),
                            data.table(importance.refr.2.ps                ,"model" = "dummy",pool = "refr", "sampling" = "ps"),
                            data.table(importance.refr.2.ad_cvar           ,"model" = "dummy",pool = "refr", "sampling" = "ad_cvar", "mtry" = 2),
                            data.table(importance.refr.2.ad_cvar_6         ,"model" = "dummy",pool = "refr", "sampling" = "ad_cvar", "mtry" = 6),
                            data.table(importance.refr.2.adfe_begin_cvar   ,"model" = "dummy",pool = "refr", "sampling" = "adfe_begin_cvar", "mtry" = 2),
                            data.table(importance.refr.2.adfe_end_cvar     ,"model" = "dummy",pool = "refr", "sampling" = "adfe_end_cvar  ", "mtry" = 2),
 use.names = TRUE )

rm(
importance.refr.1.ps              
,importance.refr.1.ad_cvar         
,importance.refr.2.ps              
,importance.refr.2.ad_cvar         
,importance.refr.2.ad_cvar_6       
,importance.refr.2.adfe_begin_cvar 
,importance.refr.2.adfe_end_cvar   
)

*list.files(outputs.path, pattern=NULL, all.files=FALSE, full.names=FALSE)*

Files include:
* outputs.path.1.refr : 11 seed x 10 rep
             oneshot,Rd,  Ad_range,Ad_sd,  Ad_coefvar: mtry in (2,6)                        
                 * test = 100
                 * nperm = 5
                 * bagging
                 * importance : ?
                 
* outputs.path.2.refr : 11 seed x 10 rep
             oneshot: mtry in (2,3,4,6) 
             Rd: mtry = 2 , (seed != 9)
             Ad_range: mtry in (2,4)  
             Ad_sd: mtry in (2,3,4,6)
             Ad_coefvar: mtry in (2,6)
             Ad_coefvar_FeatSel_Begin,  Ad_coefvar_FeatSel_End: mtry = 2                        
             * test in (100,200,300,400)
             * nperm = 5 
             * importance : all types
                      
* outputs.path.1.wrep : 10 seed x 10 rep
             oneshot,  Rd,  Ad_range,  Ad_sd,  Ad_coefvar
             * test = 100
             * nperm = 1
             * bagging
             * importance : scaled permutation
* outputs.path.2.wrep : 
            oneshot,  Rd,  Ad_range,  Ad_sd,  Ad_coefvar (all in mtry = 2)
             * test = 100
             * nperm = 1
             * importance : scaled permutation

In [ ]:
# passive sampling
